In [1]:
import pathlib
import numpy as np
import pandas as pd
import json

h = pathlib.Path.home()
pet_data = open(f"{h}/downloads/PET.txt",'r').readlines()

In [2]:
def petdata(pet_string):
    d = json.loads(pet_string)
    data_col = d['name'].replace(',','').replace('(','').replace(')','')
    values = np.array(d['data'])
    dates = values[:,0].astype(int)
    vals = values[:,1].astype(float)
    return data_col,dates,vals


In [3]:
def padd_arr(pet_string,vals_len):
    arr = petdata(pet_string)
    col_name = arr[0]
    vals = arr[2]
    if len(vals)>=vals_len:
        return col_name,vals[:vals_len]
    new_vals = np.pad(vals,(0,vals_len-len(vals)),'constant',constant_values=(0,np.nan))
    return col_name,new_vals

In [4]:
padd_arr(pet_data[700],1540)

('Rocky Mountain PADD 4 Refiner Net Production of Finished Motor Gasoline 4 Week Avg',
 array([156., 158., 151., ...,  nan,  nan,  nan]))

In [5]:
from tqdm import tqdm_notebook
data_col,dates,vals = petdata(pet_data[0])
vals_len = len(vals)
df_pet = pd.DataFrame({'date':dates,data_col:vals})

for i in tqdm_notebook(range(1,len(pet_data))):
    try:
        data_col,vals = padd_arr(pet_data[i],vals_len)
    except:
        continue
    if 'weekly' not in data_col.lower():
        continue
    if '4 week avg' in data_col.lower():
        continue
    
    df_pet[data_col] = vals

df_pet


/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


,date,U.S. Product Supplied of Residual Fuel Oil Weekly,U.S. Exports of Total Petroleum Products Weekly,U.S. Imports of Total Petroleum Products Weekly,U.S. Net Imports of Total Petroleum Products Weekly,U.S. Product Supplied of Petroleum Products Weekly,U.S. Ending Stocks excluding SPR of Crude Oil and Petroleum Products Weekly,U.S. Exports of Crude Oil and Petroleum Products Weekly,U.S. Imports of Crude Oil and Petroleum Products Weekly,U.S. Net Imports of Crude Oil and Petroleum Products Weekly,...,Midwest PADD 2 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Gulf Coast PADD 3 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,Rocky Mountain PADD 4 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,West Coast PADD 5 Ending Stocks of Reformulated RBOB with Alcohol Gasoline Blending Components Weekly,U.S. Ending Stocks of Reformulated RBOB with Ether Gasoline Blending Components Weekly,East Coast PADD 1 Ending Stocks of Reformulated RBOB with Ether Gasoline Blending Components Weekly,Midwest PADD 2 Ending Stocks of Reformulated RBOB with Ether Gasoline Blending Components Weekly,Gulf Coast PADD 3 Ending Stocks of Reformulated RBOB with Ether Gasoline Blending Components Weekly,Rocky Mountain PADD 4 RBOB with Ether Gasoline Blending Components Ending Stocks Weekly,West Coast PADD 5 Ending Stocks of Reformulated RBOB with Ether Gasoline Blending Components Weekly
0,20200807,453.0,4662.0,2302.0,-2360.0,19369.0,1446420.0,7805.0,7923.0,118.0,...,5544.0,11841.0,0.0,11518.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20200731,255.0,5259.0,2250.0,-3009.0,17912.0,1452571.0,8078.0,8260.0,182.0,...,5128.0,11693.0,0.0,12224.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20200724,471.0,4079.0,2108.0,-1971.0,19094.0,1454680.0,7290.0,7255.0,-35.0,...,5457.0,10957.0,0.0,12937.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20200717,175.0,4900.0,1923.0,-2977.0,17654.0,1461159.0,7893.0,7863.0,-30.0,...,5494.0,11788.0,0.0,13161.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20200710,383.0,4832.0,1733.0,-3099.0,18480.0,1452344.0,7375.0,7300.0,-75.0,...,5112.0,11822.0,0.0,13740.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,19910308,1145.0,1026.0,1697.0,NaN,16997.0,982108.0,1025.0,7127.0,6102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1536,19910301,1221.0,1026.0,1205.0,NaN,15414.0,987670.0,1510.0,6188.0,4678.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1537,19910222,1180.0,948.0,1287.0,NaN,15627.0,997069.0,1510.0,6221.0,4711.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1538,19910215,992.0,948.0,1046.0,NaN,16067.0,996098.0,1224.0,6573.0,5349.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_pet.to_csv('./temp_folder/df_pet.csv',index=None)

In [130]:
lower_cols = [c.lower() for c in df_pet.columns.values]
df_pet_lower = df_pet.copy()
df_pet_lower.columns = lower_cols
df_pet_lower.to_csv('./temp_folder/df_pet_lower.csv',index=None)

In [133]:
cols = df_pet_lower.columns.values
df_reports = pd.DataFrame({'report':cols[np.where(cols!='date')]})
df_reports['colnum'] = df_reports.index.values
df_reports.to_csv('./temp_folder/df_pet_columns.csv',index=False)

In [137]:
df_reports[~df_reports.report.str.contains('stocks')]

,report,colnum
0,u.s. product supplied of residual fuel oil weekly,0
1,u.s. exports of total petroleum products weekly,1
2,u.s. imports of total petroleum products weekly,2
3,u.s. net imports of total petroleum products weekly,3
4,u.s. product supplied of petroleum products weekly,4
...,...,...
1109,east coast padd 1 imports of reformulated rbob with ether gasoline blending components weekly,1109
1110,midwest padd 2 rbob with ether gasoline blending components imports weekly,1110
1111,gulf coast padd 3 imports of reformulated rbob with ether gasoline blending components weekly,1111
1112,rocky mountain padd 4 rbob with ether gasoline blending components imports weekly,1112


In [43]:
reports_text = "\n".join(df_reports.report.values)


In [44]:
print(reports_text)

U.S. Product Supplied of Residual Fuel Oil Weekly
U.S. Exports of Total Petroleum Products Weekly
U.S. Imports of Total Petroleum Products Weekly
U.S. Net Imports of Total Petroleum Products Weekly
U.S. Product Supplied of Petroleum Products Weekly
U.S. Ending Stocks excluding SPR of Crude Oil and Petroleum Products Weekly
U.S. Exports of Crude Oil and Petroleum Products Weekly
U.S. Imports of Crude Oil and Petroleum Products Weekly
U.S. Net Imports of Crude Oil and Petroleum Products Weekly
U.S. Ending Stocks of Crude Oil and Petroleum Products Weekly
U.S. Ending Stocks of Unfinished Oils Weekly
Rocky Mountain PADD 4 Ending Stocks of Reformulated Motor Gasoline Weekly
West Coast PADD 5 Ending Stocks of Reformulated Motor Gasoline Weekly
U.S. Ending Stocks of Reformulated Motor Gasoline Weekly
U.S. Imports of Total Gasoline Weekly
New England PADD 1A Ending Stocks of Total Gasoline Weekly
Central Atlantic PADD 1B Ending Stocks of Total Gasoline Weekly
Lower Atlantic PADD 1C Ending Stoc

In [46]:
import spacy
import pytextrank

# example text
text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."
text = reports_text
# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

doc = nlp(text)

# examine the top-ranked phrases in the document
for p in doc._.phrases:
    print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
#     print(p.chunks)

0.1498    20  conventional motor gasoline weekly
0.1431    34  finished motor gasoline weekly
0.1421    20  reformulated motor gasoline weekly
0.1419     6  other conventional motor gasoline weekly
0.1417     1  u.s. premium conventional retail gasoline prices weekly
0.1406     1  midgrade conventional retail gasoline prices weekly
0.1405    16  total gasoline weekly
0.1405     1  padd 1c midgrade conventional retail gasoline prices weekly
0.1394     1  premium reformulated retail gasoline prices weekly
0.1390     1  u.s. midgrade conventional retail gasoline prices weekly
0.1383     1  gulf coast premium conventional retail gasoline prices weekly
0.1382     1  east coast premium conventional retail gasoline prices weekly
0.1379     1  u.s. premium reformulated retail gasoline prices weekly
0.1377     6  other finished conventional motor gasoline weekly
0.1375     4  gasoline blending components weekly
0.1368     1  texas premium conventional retail gasoline prices weekly
0.1367     1 

In [50]:
ranks = []
counts = []
texts = []
for p in doc._.phrases:
    ranks.append(p.rank)
    counts.append(p.count)
    texts.append(p.text)



In [63]:
df_phrases_byrank = pd.DataFrame({'p_rank':ranks,'p_count':counts,'p_text':texts})
df_phrases_bycount = df_phrases_byrank.sort_values('p_count',ascending=False)


In [52]:
pd.set_option('display.max_colwidth', 0)
df_phrases_byrank

,p_rank,p_count,p_text
0,0.149850,20,conventional motor gasoline weekly
1,0.143140,34,finished motor gasoline weekly
2,0.142084,20,reformulated motor gasoline weekly
3,0.141934,6,other conventional motor gasoline weekly
4,0.141683,1,u.s. premium conventional retail gasoline prices weekly
...,...,...,...
657,0.010961,2,massachusetts
658,0.010441,4,1 percent
659,0.010129,7,all grades
660,0.006872,6,all countries


In [73]:
import dash_table
import dash
import dash_html_components as html

def run_reports(df,displayed_rows=100):

#     displayed_rows =10
    dt_reports = dash_table.DataTable(
        data=df.to_dict('records'),
        columns = [{'name':c,'id':c} for c in df.columns.values],
        page_size=displayed_rows,
        page_action='native',
        style_table={
            'overflowY':'auto',
#             'height': f'{35*(displayed_rows+1)+1}px'
        },
        style_cell={
            'whiteSpace': 'normal',
            'height': 'auto',
            'minWidth': '50px'
        },
        filter_action='native',
#         fixed_rows={'headers': True},
    )
    app = dash.Dash()
    s = {'display':'grid'}
    app.layout=html.Div(
        [dt_reports],
        style=s
    )
    app.run_server()
        

In [74]:
run_reports(df_phrases_bycount[['p_count','p_text']])

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 i

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Aug/2020 11:01:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 11:01:12] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 11:01:12] "GET /_dash-layout HTTP/1.1" 200 -


In [75]:
df_phrases_byrank.to_csv(f'{h}/downloads/df_phrases_byrank.csv',index=False)

In [90]:
import re
s = "padd.{1,2}[1-5][a-e]{0,1}"
df2 = df_phrases_byrank[df_phrases_byrank.p_text.str.contains(s)]
finds = [re.findall(s,v)[0] for v  in df2.p_text.values]
splits = [re.split(s,v) for v  in df2.p_text.values]
df2['token'] = finds
df2['left'] = [s[0] for s in splits]
df2['right'] = [None if len(s)<2 else s[1] for s in splits]

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [91]:
df2

,p_rank,p_count,p_text,token,left,right
7,0.140474,1,padd 1c midgrade conventional retail gasoline prices weekly,padd 1c,,midgrade conventional retail gasoline prices weekly
38,0.132317,1,central atlantic padd 1b midgrade conventional retail gasoline prices weekly,padd 1b,central atlantic,midgrade conventional retail gasoline prices weekly
41,0.131423,1,central atlantic padd 1b premium reformulated retail gasoline prices weekly,padd 1b,central atlantic,premium reformulated retail gasoline prices weekly
61,0.125232,1,central atlantic padd 1b propane residential price weekly,padd 1b,central atlantic,propane residential price weekly
92,0.124412,1,lower atlantic padd 1c propane residential price weekly,padd 1c,lower atlantic,propane residential price weekly
...,...,...,...,...,...,...
445,0.053555,1,midwest padd 2 percent,padd 2,midwest,percent
447,0.053202,1,padd 1a premium,padd 1a,,premium
461,0.050503,1,padd 3 propane and propylene ending stocks,padd 3,,propane and propylene ending stocks
462,0.050296,1,rocky mountain padd 4 imports of other oils excluding,padd 4,rocky mountain,imports of other oils excluding


In [97]:
cols = ['california regular reformulated retail gasoline prices weekly','california regular reformulated retail gasoline prices']
df_pet2 = df_pet.copy()
df_pet2.columns = [v.lower()for v in df_pet2.columns.values]

In [98]:
df_pet2[cols]

KeyError: "['california regular reformulated retail gasoline prices'] not in index"

In [125]:
cols = [c for c in df_pet.columns.values if len(re.findall('Regular Reformulated Retail Gasoline Prices Weekly',c))>0]
# cols = [c for c in cols if 'except' not in c.lower()]
# cols = [c for c in cols if 'all' not in c.lower()]

In [126]:
cols

['Boston MA Regular Reformulated Retail Gasoline Prices Weekly',
 'Chicago IL Regular Reformulated Retail Gasoline Prices Weekly',
 'California Regular Reformulated Retail Gasoline Prices Weekly',
 'West Coast PADD 5 Except California Regular Reformulated Retail Gasoline Prices Weekly',
 'Los Angeles CA Regular Reformulated Retail Gasoline Prices Weekly',
 'San Francisco CA Regular Reformulated Retail Gasoline Prices Weekly',
 'New York Harbor Regular Reformulated Retail Gasoline Prices Weekly',
 'Houston TX Regular Reformulated Retail Gasoline Prices Weekly',
 'U.S. Regular Reformulated Retail Gasoline Prices Weekly',
 'East Coast Regular Reformulated Retail Gasoline Prices Weekly',
 'New England PADD 1A Regular Reformulated Retail Gasoline Prices Weekly',
 'Central Atlantic PADD 1B Regular Reformulated Retail Gasoline Prices Weekly',
 'Lower Atlantic PADD 1C Regular Reformulated Retail Gasoline Prices Weekly',
 'Midwest Regular Reformulated Retail Gasoline Prices Weekly',
 'Gulf Coas

In [128]:
df_pet[['date']+cols]

,date,Boston MA Regular Reformulated Retail Gasoline Prices Weekly,Chicago IL Regular Reformulated Retail Gasoline Prices Weekly,California Regular Reformulated Retail Gasoline Prices Weekly,West Coast PADD 5 Except California Regular Reformulated Retail Gasoline Prices Weekly,Los Angeles CA Regular Reformulated Retail Gasoline Prices Weekly,San Francisco CA Regular Reformulated Retail Gasoline Prices Weekly,New York Harbor Regular Reformulated Retail Gasoline Prices Weekly,Houston TX Regular Reformulated Retail Gasoline Prices Weekly,U.S. Regular Reformulated Retail Gasoline Prices Weekly,East Coast Regular Reformulated Retail Gasoline Prices Weekly,New England PADD 1A Regular Reformulated Retail Gasoline Prices Weekly,Central Atlantic PADD 1B Regular Reformulated Retail Gasoline Prices Weekly,Lower Atlantic PADD 1C Regular Reformulated Retail Gasoline Prices Weekly,Midwest Regular Reformulated Retail Gasoline Prices Weekly,Gulf Coast Regular Reformulated Retail Gasoline Prices Weekly,West Coast Regular Reformulated Retail Gasoline Prices Weekly,Massachusetts Regular Reformulated Retail Gasoline Prices Weekly,New York Regular Reformulated Retail Gasoline Prices Weekly,Texas Regular Reformulated Retail Gasoline Prices Weekly
0,20200807,2.082,2.353,3.054,2.353,3.028,3.176,2.099,1.805,2.354,2.121,2.102,2.164,1.980,2.263,1.782,2.953,2.088,2.112,1.782
1,20200731,2.083,2.345,3.058,2.364,3.041,3.188,2.105,1.831,2.370,2.130,2.107,2.173,1.996,2.266,1.846,2.958,2.090,2.128,1.846
2,20200724,2.084,2.359,3.049,2.366,3.033,3.156,2.119,1.842,2.369,2.136,2.108,2.182,2.002,2.281,1.845,2.950,2.089,2.128,1.845
3,20200717,2.088,2.370,3.031,2.372,3.015,3.102,2.125,1.855,2.373,2.136,2.113,2.180,2.001,2.291,1.880,2.936,2.094,2.140,1.880
4,20200710,2.078,2.398,3.022,2.366,3.007,3.092,2.135,1.853,2.372,2.137,2.110,2.184,2.001,2.312,1.867,2.928,2.086,2.152,1.867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,19910308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1536,19910301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1537,19910222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1538,19910215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
